In [1]:
import pandas as pd

from pandas.api.types import CategoricalDtype

# Define the specific order for adapter_config_string
# adapter_order = ['full_tuning', 'houlsby', 'pfeiffer', 'scaled_parallel', 'ln_tuning',
#                  'lora', 'hf_lora_all', 'hf_krona', 'compacter', 'compacter++', 'ia3',
#                  'prefix_tuning[bottleneck_size=800,kv_size=64]|par_bn', 'prefix_tuning[kv_size=64]', 'prefix_tuning_flat[kv_size=64]', 'lora[r=8,use_gating=True]|prefix_tuning[prefix_length=10,use_gating=True,kv_size=64]|seq_bn[reduction_factor=16,use_gating=True]']
adapter_order = ['full_tuning', 'houlsby', 'pfeiffer', 'scaled_parallel', 'ln_tuning',
                 'lora', 'hf_lora', 'hf_lora_all', 'hf_krona', 'compacter', 'compacter++', 'ia3',
                 'mam', 'prefix_tuning', 'prefix_tuning_flat', 'unipelt']
adapter_order = adapter_order[::-1]


In [2]:
df = pd.read_csv("../wandb_results/wandb_export_2023-12-22T05_29_37.476-05_00.csv")

relevant_columns = ['model_name_or_path', 'adapter_config_string', 'total_parameters',
                    'trainable_parameters', 'throughput_examples', 'test/throughput_examples',
                    'total_batch_size', 'per_device_train_batch_size', 'gradient_accumulation_steps',
                    'dataset_name', 'dataset_config_name', 'Name']

df = df[relevant_columns]
df = df.dropna()
# df = df[df.model_name_or_path != "t5-11b"]

df['adapter_config_string'] = df['adapter_config_string'].replace(
    "prefix_tuning[bottleneck_size=800,kv_size=64]|par_bn", "mam"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "prefix_tuning[bottleneck_size=800,kv_size=128]|par_bn", "mam"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "prefix_tuning_flat[kv_size=128]", "prefix_tuning_flat"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "prefix_tuning_flat[kv_size=64]", "prefix_tuning_flat"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "prefix_tuning[kv_size=128]", "prefix_tuning"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "prefix_tuning[kv_size=64]", "prefix_tuning"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "lora[r=8,use_gating=True]|prefix_tuning[prefix_length=10,use_gating=True,kv_size=128]|seq_bn[reduction_factor=16,use_gating=True]", "unipelt"
)
df['adapter_config_string'] = df['adapter_config_string'].replace(
    "lora[r=8,use_gating=True]|prefix_tuning[prefix_length=10,use_gating=True,kv_size=64]|seq_bn[reduction_factor=16,use_gating=True]", "unipelt"
)

# num_gpus = total_batch_size / (per_device_train_batch_size * gradient_accumulation_steps)
df = df.assign(num_gpus=df['total_batch_size'] / (
        df['per_device_train_batch_size'] * df['gradient_accumulation_steps']))

# throughput_examples_per_gpu = throughput_examples / num_gpus
df = df.assign(
    throughput_examples_per_gpu=df['throughput_examples'] / df['num_gpus'])

# test/throughput_examples
df = df.assign(
    throughput_examples_per_gpu_test=df['test/throughput_examples'] / df['num_gpus'])

table = df.groupby(
    ['model_name_or_path', 'adapter_config_string', 'num_gpus'])\
    .agg({
        'total_parameters': 'mean',
        'trainable_parameters': 'mean',
        'throughput_examples_per_gpu': ['mean', 'std'],
        'throughput_examples_per_gpu_test': ['mean', 'std'],
        'Name': 'count',
    }).reset_index()

table['adapter_config_string'] = table['adapter_config_string'].astype(
    CategoricalDtype(categories=adapter_order, ordered=True)
)
table = table.sort_values(
    by=['model_name_or_path', 'num_gpus', 'adapter_config_string'],
    ascending=[False, True, False]
)

print("Unique models:", len(table['model_name_or_path'].unique()))
print("Unique adapter configs:", len(table['adapter_config_string'].unique()))

print(table.shape)
table.to_csv('../wandb_results/throughput.csv', index=False)
table

Unique models: 3
Unique adapter configs: 16
(48, 10)


model_name_or_path adapter_config_string num_gpus total_parameters  \
                                                                 mean   
34           t5-large           full_tuning      1.0     7.376681e+08   
38           t5-large               houlsby      1.0     7.832545e+08   
43           t5-large              pfeiffer      1.0     7.769108e+08   
46           t5-large       scaled_parallel      1.0     8.209726e+08   
40           t5-large             ln_tuning      1.0     7.376681e+08   
41           t5-large                  lora      1.0     7.729265e+08   
36           t5-large               hf_lora      1.0     7.400274e+08   
37           t5-large           hf_lora_all      1.0     7.463188e+08   
35           t5-large              hf_krona      1.0     7.378893e+08   
32           t5-large             compacter      1.0     7.708714e+08   
33           t5-large           compacter++      1.0     7.707193e+08   
39           t5-large                   ia3      1.0     7.709112e+08   
42           t5-large                   mam      1.0     9.416370e+08   
44           t5-large         prefix_tuning      1.0     8.478787e+08   
45           t5-large    prefix_tuning_flat      1.0     7.749908e+08   
47           t5-large               unipelt      1.0     8.567908e+08   
22              t5-3b               houlsby      1.0     2.897185e+09   
27              t5-3b              pfeiffer      1.0     2.890841e+09   
30              t5-3b       scaled_parallel      1.0     2.934903e+09   
24              t5-3b             ln_tuning      1.0     2.851598e+09   
25              t5-3b                  lora      1.0     2.890396e+09   
20              t5-3b               hf_lora      1.0     2.857497e+09   
21              t5-3b           hf_lora_all      1.0     2.876764e+09   
19              t5-3b              hf_krona      1.0     2.852004e+09   
17              t5-3b             compacter      1.0     2.884802e+09   
18              t5-3b           compacter++      1.0     2.884650e+09   
23              t5-3b                   ia3      1.0     2.885874e+09   
26              t5-3b                   mam      1.0     3.409904e+09   
28              t5-3b         prefix_tuning      1.0     3.188744e+09   
29              t5-3b    prefix_tuning_flat      1.0     2.902192e+09   
31              t5-3b               unipelt      1.0     3.201195e+09   
5              t5-11b               houlsby      1.0     1.135291e+10   
11             t5-11b              pfeiffer      1.0     1.134656e+10   
15             t5-11b       scaled_parallel      1.0     1.139063e+10   
8              t5-11b             ln_tuning      1.0     1.130732e+10   
10             t5-11b                  lora      1.0     1.136027e+10   
3              t5-11b               hf_lora      1.0     1.132738e+10   
4              t5-11b           hf_lora_all      1.0     1.139855e+10   
2              t5-11b              hf_krona      1.0     1.130810e+10   
0              t5-11b             compacter      1.0     1.134052e+10   
1              t5-11b           compacter++      1.0     1.134037e+10   
7              t5-11b                   ia3      1.0     1.134573e+10   
13             t5-11b         prefix_tuning      1.0     1.255221e+10   
14             t5-11b    prefix_tuning_flat      1.0     1.141100e+10   
16             t5-11b               unipelt      1.0     1.257881e+10   
6              t5-11b               houlsby      8.0     1.135291e+10   
12             t5-11b              pfeiffer      8.0     1.134656e+10   
9              t5-11b             ln_tuning      8.0     1.130732e+10   

   trainable_parameters throughput_examples_per_gpu             \
                   mean                        mean        std   
34         7.376681e+08                  214.719403  55.264638   
38         1.268736e+07                  138.443747  10.812014   
43         6.343680e+06                  122.391611  31.848107   
46         5.040542e+0

In [140]:
table["Name"]["count"].sum()

112